In [1]:
import csv
import pandas as pd
import numpy as np
import re 
import spacy

from tqdm import tqdm
from spacy.tokens import DocBin
from sklearn.model_selection import train_test_split

# ebablbe auto-completion
%config Completer.use_jedi = False

## Generate Config File

download model.

In [2]:
# !python -m spacy download en_core_web_trf

generate base_config.cfg and base_config_gpu.cfg on https://spacy.io/usage/training#quickstart

In [3]:
# !python -m spacy init fill-config base_config.cfg config.cfg
# !python -m spacy init fill-config base_config_gpu.cfg config_gpu.cfg

## Gather Data

###  Convert Excel to Txt
upload the excel file to http://15.15.166.35:18888/tree/eipi10/jian-xu3/snps-classification/snps/etl/data/case_note_ner/xlsx

~~~shell
!pip install xlrd==1.2.0
~~~

In [4]:
# !ls -l /tf/eipi10/jian-xu3/snps-classification/snps/etl/data/case_note_ner/xlsx/ 

In [5]:
# ! /tf/eipi10/jian-xu3/snps-classification/snps/etl/xlsx2txt.py  case_note_ner '' 1 '\t'

### Load Txt File

In [6]:

def load_data(text_file):
    names = ['Case_ID', 'Work_Order_Number', 'Created_On', 'Service_Account', 'Due_Date', 
             'Action_Booking_status', 'Action_Date', 'Currently_Worked_by', 'Owner', 'Product_Line', 
             'Follow_Up_Reason_Code', 'Response_Time', 'Work_Order_Type', 'Postal_Code', 'Active_Booking', 
             'Follow_Up_Type', 'Business_Segment', 'Follow_Up_Note', 'Reschedule_Note']
    dtypes = [np.str, np.str, np.str, np.str, np.str, 
              np.str, np.str, np.str, np.str, np.str, 
              np.str, np.str, np.str, np.str, np.str,
              np.str, np.str, np.str, np.str]
    dtypes = {name:dtype for name, dtype in zip(names, dtypes)}
    df = pd.read_table(text_file, names=names, dtype=dtypes, quoting=csv.QUOTE_NONE)

    df.index = range(1, len(df)+1) 
    df['id'] = df.index
      
    return df

text_file='/tf/eipi10/jian-xu3/snps-classification/snps/etl/data/case_note_ner/txt/3RS_July_Extraction_with_notes_004.txt'
df_case = load_data(text_file)

In [7]:
print('-'*50)
print(len(df_case))

print('-'*50)
display(df_case.head(5))

print('-'*50)
display(df_case.describe().T)

print('-'*50)
df_case.info()



--------------------------------------------------
7039
--------------------------------------------------


,Case_ID,Work_Order_Number,Created_On,Service_Account,Due_Date,Action_Booking_status,Action_Date,Currently_Worked_by,Owner,Product_Line,Follow_Up_Reason_Code,Response_Time,Work_Order_Type,Postal_Code,Active_Booking,Follow_Up_Type,Business_Segment,Follow_Up_Note,Reschedule_Note,id
1,5060815932,WO-012952625,2021-02-03 10:38:00,William Torres,2021-04-26 15:00:00,Additional Parts Required,2021-07-03 11:45:00,Kerby Singleton,Nancy Quesada,Commercial Desktop PC Premium L6,NaN,NCD,Break Fix,30305,PC-Kerby Singleton,Additional Parts Required,Computing,NaN,#NORMAL Part escalated,1
2,5060815932,WO-012952625,2021-02-03 10:38:00,William Torres,2021-04-26 15:00:00,Additional Parts Required,2021-07-03 11:45:00,Kerby Singleton,Nancy Quesada,Commercial Desktop PC Premium L6,NaN,NCD,Break Fix,30305,PC-Kerby Singleton,Additional Parts Required,Computing,NaN,Please cancel backordered part. Customer has c...,2
3,5061272897,WO-013009621,2021-02-09 13:39:00,LAS VEGAS METRO POLICE DEPT,2021-02-24 19:00:00,Additional Parts Required,2021-07-16 15:30:00,NaN,Avinash Shukla,Commercial Premium Notebooks,NaN,NCD,Break Fix,89101,PC-Joe Walper,Additional Parts Required,Computing,NaN,According to bryan this laptop shows repair co...,3
4,5061272897,WO-013009621,2021-02-09 13:39:00,LAS VEGAS METRO POLICE DEPT,2021-02-24 19:00:00,Additional Parts Required,2021-07-16 15:30:00,NaN,Avinash Shukla,Commercial Premium Notebooks,NaN,NCD,Break Fix,89101,PC-Joe Walper,Additional Parts Required,Computing,NaN,Back order part,4
5,5061278521,WO-013010632-4,2021-06-16 12:17:00,MR. REGINALD JOSEPH,2021-07-15 14:42:00,Reschedule,2021-07-15 13:09:00,NaN,JOCILYN ANDREA SANDI DURAN,Commercial Premium Notebooks,NaN,NCD,Break Fix,20785,USA-PR-USYWWDWN_E_A_EEG,Reschedule,Computing,NaN,NaN,5


--------------------------------------------------


,count,mean,std,min,25%,50%,75%,max
id,7039.0,3520.0,2032.128605,1.0,1760.5,3520.0,5279.5,7039.0


--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7039 entries, 1 to 7039
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Case_ID                7039 non-null   object
 1   Work_Order_Number      7039 non-null   object
 2   Created_On             7039 non-null   object
 3   Service_Account        7038 non-null   object
 4   Due_Date               7017 non-null   object
 5   Action_Booking_status  4223 non-null   object
 6   Action_Date            3408 non-null   object
 7   Currently_Worked_by    887 non-null    object
 8   Owner                  7039 non-null   object
 9   Product_Line           6457 non-null   object
 10  Follow_Up_Reason_Code  2864 non-null   object
 11  Response_Time          7039 non-null   object
 12  Work_Order_Type        7039 non-null   object
 13  Postal_Code            7039 non-null   object
 14  Active_Booking       

In [8]:
print(df_case.groupby(['Follow_Up_Type']).size())

Follow_Up_Type
Additional Parts Required     1666
Onsite Stock Replenishment     376
Partner Request Reassign        16
Request Reassign              2100
Reschedule                     434
Revisit                       2447
dtype: int64


### Data Cleaning

In [9]:
def filter_out_note_na(df):
    df_filter = df.loc[(df.Follow_Up_Note.notna()) | (df.Reschedule_Note.notna())].copy()
    return df_filter

df_train_test = filter_out_note_na(df_case)
print(len(df_train_test))

# df_train_test['notes'] = [('' if pd.isna(fun) else fun)  + ('' if pd.isna(crn) else ' | ' + crn)
#                           for fun, crn in zip(df_train_test.Follow_Up_Note, df_train_test.Reschedule_Note)] 
df_train_test['notes'] = [(crn if pd.isna(fun) else (fun if pd.isna(crn) else fun + '.' + crn))
                          for fun, crn in zip(df_train_test.Follow_Up_Note, df_train_test.Reschedule_Note)] 
df_train_test

6110


,Case_ID,Work_Order_Number,Created_On,Service_Account,Due_Date,Action_Booking_status,Action_Date,Currently_Worked_by,Owner,Product_Line,...,Response_Time,Work_Order_Type,Postal_Code,Active_Booking,Follow_Up_Type,Business_Segment,Follow_Up_Note,Reschedule_Note,id,notes
1,5060815932,WO-012952625,2021-02-03 10:38:00,William Torres,2021-04-26 15:00:00,Additional Parts Required,2021-07-03 11:45:00,Kerby Singleton,Nancy Quesada,Commercial Desktop PC Premium L6,...,NCD,Break Fix,30305,PC-Kerby Singleton,Additional Parts Required,Computing,NaN,#NORMAL Part escalated,1,#NORMAL Part escalated
2,5060815932,WO-012952625,2021-02-03 10:38:00,William Torres,2021-04-26 15:00:00,Additional Parts Required,2021-07-03 11:45:00,Kerby Singleton,Nancy Quesada,Commercial Desktop PC Premium L6,...,NCD,Break Fix,30305,PC-Kerby Singleton,Additional Parts Required,Computing,NaN,Please cancel backordered part. Customer has c...,2,Please cancel backordered part. Customer has c...
3,5061272897,WO-013009621,2021-02-09 13:39:00,LAS VEGAS METRO POLICE DEPT,2021-02-24 19:00:00,Additional Parts Required,2021-07-16 15:30:00,NaN,Avinash Shukla,Commercial Premium Notebooks,...,NCD,Break Fix,89101,PC-Joe Walper,Additional Parts Required,Computing,NaN,According to bryan this laptop shows repair co...,3,According to bryan this laptop shows repair co...
4,5061272897,WO-013009621,2021-02-09 13:39:00,LAS VEGAS METRO POLICE DEPT,2021-02-24 19:00:00,Additional Parts Required,2021-07-16 15:30:00,NaN,Avinash Shukla,Commercial Premium Notebooks,...,NCD,Break Fix,89101,PC-Joe Walper,Additional Parts Required,Computing,NaN,Back order part,4,Back order part
6,5064053283,WO-013536670-2,2021-06-30 11:06:00,Ford Motor Company of Canada,2021-07-02 08:30:00,Request Reassign,2021-07-05 14:45:00,NaN,Gabriel Bustos,A3 Laserjet,...,NCD,IMACD,N9A,Rick Harbarenko,Request Reassign,dMPS,NaN,Please send this case to Corey Harbarenko. Cor...,6,Please send this case to Corey Harbarenko. Cor...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7035,5069292833,WO-014560602,2021-07-16 16:23:00,Kaiser Foundation Health Plan Inc.,2021-07-19 15:09:00,Additional Parts Required,2021-07-16 16:00:00,NaN,6 CSPAE166,A4 PageWide SMB & Enterprise managed,...,Not available,Break Fix,91786,Anthony Garcia,Additional Parts Required,pMPS,NaN,Please order the following part. Qty. 1 B5L04...,7035,Please order the following part. Qty. 1 B5L04...
7036,5069293232,WO-014560595,2021-07-16 16:21:00,CDW - St. Lukes Hospital (Government),2021-07-20 16:26:00,Request Reassign,NaN,NaN,1 CSPAE131,A4 Laserjet SMB,...,Not available,Break Fix,55805,Brock Tharp,Request Reassign,pMPS,NaN,Please reassign to lms,7036,Please reassign to lms
7037,5069293974,WO-014560699,2021-07-16 17:06:00,LITHIA NISSAN OF FRESNO,2021-07-19 18:00:00,Request Reassign,2021-07-19 13:00:00,Aaron Stavinsky,Luis Alejandro Vargas,A4 Laserjet Enterprise,...,NCD,Preventative Maintenance,93710,IS-Aaron Stavinsky,Request Reassign,Printing,NaN,dahlia.garcia@hp.com,7037,dahlia.garcia@hp.com
7038,5069294340,WO-014560697,2021-07-16 17:04:00,"Best Buy co., Inc.",2021-07-20 18:00:00,NaN,NaN,NaN,Subir Das,Retail Solutions Core,...,NCD,Break Fix,95678,PR-David Popadiuc,Revisit,Computing,Please order 842275-001 and have it ship to HF...,NaN,7038,Please order 842275-001 and have it ship to HF...


In [10]:
print(df_train_test.groupby(['Follow_Up_Type']).size())

Follow_Up_Type
Additional Parts Required     1579
Onsite Stock Replenishment     375
Partner Request Reassign        13
Request Reassign              1737
Reschedule                     136
Revisit                       2270
dtype: int64


### Generate NER Label

In [11]:
def standardize_text1(text):
    text = text.strip().lower()
    # add space between punctuation
    text = re.sub(r'([.\\!?,\'()\[\]"|;])', r' \1 ', text)
#     # remove characters except for English letters and some punctuations
#     text = re.sub(r"[^A-Za-z\.\-\?\!\,\#\@\% ]", "", text)
    # remove extra spaces
    text = re.sub(r'[" "]+', " ", text)
    return text

In [12]:
nlp = spacy.load("en_core_web_sm")
notes = [standardize_text1(note) for note in df_train_test.notes]
docs = nlp.pipe(notes)

In [13]:
def get_samples(df):
    df_filter = df.loc[(df.Work_Order_Number=='WO-013865844-1') | 
                       (df.Work_Order_Number=='WO-013910857') |
                       (df.Work_Order_Number=='WO-014466680') |
                       (df.Work_Order_Number=='WO-014466875') |
                       (df.Work_Order_Number=='WO-014466875-1') 
                      ].copy()
    return df_filter

df_sample = get_samples(df_train_test)
df_sample

,Case_ID,Work_Order_Number,Created_On,Service_Account,Due_Date,Action_Booking_status,Action_Date,Currently_Worked_by,Owner,Product_Line,...,Response_Time,Work_Order_Type,Postal_Code,Active_Booking,Follow_Up_Type,Business_Segment,Follow_Up_Note,Reschedule_Note,id,notes
21,5065852054,WO-013865844-1,2021-07-02 14:10:00,KAISER PERMANENTE,2021-07-16 18:00:00,NaN,NaN,Ryan Bartoli,Ezequiel Jara Corao,Detachables,...,NCD,Break Fix,94553,CT-Ryan Bartoli,Revisit,Computing,Please create a revisit for Wednesday,#NORMAL Please order part # L51580-001 and L31...,21,Please create a revisit for Wednesday.#NORMAL ...
29,5066093878,WO-013910857,2021-05-14 17:16:00,AMD,2021-07-02 17:00:00,NaN,NaN,NaN,JORGE DANIEL BENAVIDEZ AVILA,Commercial Premium Notebooks,...,NCD,Break Fix,78735,PR-Rob Silcocks,Revisit,Computing,Please order battery 933321-855 ship to UPS H...,Please reassign to Rob Silcocks,29,Please order battery 933321-855 ship to UPS H...
2978,5068832635,WO-014466680,2021-07-08 10:24:00,Jabil circuit Inc,2021-07-09 17:00:00,Additional Parts Required,2021-07-15 08:00:00,NaN,7 CSPAE147,Large Format Design Pagewide XL HW,...,NCD,Break Fix,87106,PR-Isaac Lucero,Additional Parts Required,dMPS,NaN,"Per L2 please order: 2YB64-67005, 2YB64-67003 ...",2978,"Per L2 please order: 2YB64-67005, 2YB64-67003 ..."
3025,5068836700,WO-014466875,2021-07-08 10:49:00,Border Patrol and Customs,2021-07-14 16:00:00,NaN,NaN,NaN,Abu Ambia,Large Format Design HW,...,NCD,Break Fix,79925,PR-Manuel Bailon,Revisit,Printing,Please create a revisite for 7-16-2021 at 1300...,NaN,3025,Please create a revisite for 7-16-2021 at 1300...
3026,5068836700,WO-014466875-1,2021-07-14 15:01:00,Border Patrol and Customs,2021-07-16 16:00:00,NaN,NaN,NaN,Lisbeth Alejandra Sibaja,Large Format Design HW,...,NCD,Break Fix,79925,PR-Manuel Bailon,Revisit,Printing,Please create a revisite for 7-23-2021 for pri...,NaN,3026,Please create a revisite for 7-23-2021 for pri...


In [14]:

sample_notes = [standardize_text1(note) for note in df_sample.notes]
sampel_docs = nlp.pipe(sample_notes)
for i, doc in enumerate(sampel_docs):
    print('-'*50)
    print(doc.text)
    print([(token.text, token.pos_) for token in doc])
    print([(ent.text, ent.label_) for ent in doc.ents]) 

--------------------------------------------------
please create a revisit for wednesday . #normal please order part # l51580-001 and l31365-001 to concord , california hal setting up time to meet with customer . 
[('please', 'INTJ'), ('create', 'VERB'), ('a', 'DET'), ('revisit', 'NOUN'), ('for', 'ADP'), ('wednesday', 'PROPN'), ('.', 'PUNCT'), ('#', 'NOUN'), ('normal', 'ADJ'), ('please', 'INTJ'), ('order', 'VERB'), ('part', 'NOUN'), ('#', 'SYM'), ('l51580', 'PROPN'), ('-', 'PUNCT'), ('001', 'NUM'), ('and', 'CCONJ'), ('l31365', 'NOUN'), ('-', 'PUNCT'), ('001', 'NUM'), ('to', 'ADP'), ('concord', 'PROPN'), (',', 'PUNCT'), ('california', 'PROPN'), ('hal', 'NOUN'), ('setting', 'VERB'), ('up', 'ADP'), ('time', 'NOUN'), ('to', 'PART'), ('meet', 'VERB'), ('with', 'ADP'), ('customer', 'NOUN'), ('.', 'PUNCT')]
[('wednesday', 'DATE'), ('# l51580-001', 'MONEY'), ('concord', 'GPE'), ('california', 'GPE')]
--------------------------------------------------
please order battery 933321-855 ship to ups

In [15]:
patterns = [
 {'expression': r'[a-zA-Z]\d\d\d\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d\d\d-[a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-\d[a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d\d\d-[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-[a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]-\d\d\d\d\d-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d-\d\d\d\d-\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d\d-\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d[a-zA-Z][a-zA-Z]\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d[a-zA-Z]\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d\d\d-[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d-\d\d\d-\d\d\d-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]-\d\d\d\d-\d\d\d-\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d\d-[a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d-\d\d\d\d[a-zA-Z]-\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d-\d\d\d\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d-\d\d\d\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d-\d\d\d\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d-\d\d\d-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d[a-zA-Z][a-zA-Z]\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d\d-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d\d-[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d[a-zA-Z]-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d[a-zA-Z]-[a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d-\d\d\d\d\d-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d[a-zA-Z]\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z]\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z]\d\d[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d[a-zA-Z][a-zA-Z]\d\d[a-zA-Z]\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d[a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d-\d\d\d\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d[a-zA-Z]\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d[a-zA-Z]\d\d[a-zA-Z]-[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z]\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d\d\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d[a-zA-Z][a-zA-Z]\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z]\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z]-[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d-\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d[a-zA-Z][a-zA-Z]\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d-\d\d\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d-\d\d-[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z]\d[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z]\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z][a-zA-Z]\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d[a-zA-Z]\d\d[a-zA-Z][a-zA-Z]\d[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z][a-zA-Z]\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d-[a-zA-Z]\d[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d[a-zA-Z][a-zA-Z]\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d[a-zA-Z][a-zA-Z]\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z]\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d[a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d-[a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d[a-zA-Z]\d-\d\d\d\d-\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d\d-\d\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z]\d[a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d-\d\d\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d-[a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d-[a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d[a-zA-Z]\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d-\d\d\d\d-\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d\d-\d\d\d\d\d-[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d[a-zA-Z][a-zA-Z]\d\d[a-zA-Z]-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d[a-zA-Z][a-zA-Z]-\d\d\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d-\d\d\d\d-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d-\d\d\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d-\d\d-\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d\d\d\d-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z]\d[a-zA-Z]-\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d\d[a-zA-Z]\d\d-\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d[a-zA-Z]\d\d[a-zA-Z][a-zA-Z]-\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d-\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d[a-zA-Z]\d\d[a-zA-Z]\d-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d\d\d\d-\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d-\d\d\d\d-\d\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d[a-zA-Z]\d\d-\d\d\d\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d\d\d-[a-zA-Z]\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d-[a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d[a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d[a-zA-Z]-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
#  {'expression': r'order', 'label': 'ACTION'}, 
#  {'expression': r'recommended', 'label': 'ACTION'}, 
#  {'expression': r'hfpu', 'label': 'ACTION'}, 
#  {'expression': r'ship', 'label': 'ACTION'}, 
#  {'expression': r'fedex', 'label': 'ACTION'}, 
#  {'expression': r'hal', 'label': 'ACTION'}, 
#  {'expression': r'site', 'label': 'ACTION'}, 
#  {'expression': r'send', 'label': 'ACTION'}, 
#  {'expression': r'location', 'label': 'ACTION'}, 
#  {'expression': r'pickup', 'label': 'ACTION'}, 
#  {'expression': r'sent', 'label': 'ACTION'}, 
#  {'expression': r'requested', 'label': 'ACTION'}, 
#  {'expression': r'reorder', 'label': 'ACTION'}, 
#  {'expression': r'street', 'label': 'ACTION'}, 
#  {'expression': r'hfp', 'label': 'ACTION'}, 
#  {'expression': r'shipped', 'label': 'ACTION'}, 
#  {'expression': r'delivery', 'label': 'ACTION'}, 
#  {'expression': r'recommend', 'label': 'ACTION'}, 
#  {'expression': r'warehouse', 'label': 'ACTION'}    
]

In [16]:
def generate_label(texts, patterns={}):   
    
    def entity_from_patterns(text, patterns, pattern_matchs):        
        entities = {}
#         print('-'*50)  
        for pattern in patterns:
            entities = entity_from_pattern(text, pattern, entities, pattern_matchs)
        entities = [value for _, value in sorted(entities.items(), key=lambda item: item[0][0])]
        return entities

    def entity_from_pattern(text, pattern, entities, pattern_matchs):
        expression = pattern['expression']
        label = pattern['label']
        for match in re.finditer(expression, text):
            start, end = match.span()    
            add_key = True
            if start-1 >= 0 and text[start-1]!=' ':
                add_key = False
            if end < len(text) and text[end]!=' ':
                add_key = False                
                
#             drop_keys = []
#             for key, value in entities.items():
#                 start_, end_ = key
#                 if start<start_ and end>end_:
#                     drop_keys.append(key)
#                 if start>start_ and end<end_:
#                     add_key = False
#             for key in drop_keys:
#                 entities.pop(key)

            if add_key: 
#                 print(text, expression)
                pattern_matchs[expression] = pattern_matchs[expression] + 1
                entities[(start, end)] = (start, end, label) 
        return entities
        
    examples = []
    pattern_matchs = {pattern['expression']:0 for pattern in patterns}
    for text in texts:
        entities = entity_from_patterns(text, patterns, pattern_matchs)        
        example = (text, {"entities": entities})
        examples.append(example)
    pattern_matchs = [(key, value) for key, value in sorted(pattern_matchs.items(), key=lambda item: -item[1]) if value>0]
#     pattern_matchs = [(key, value) for key, value in pattern_matchs if value>0]
        
    return examples, pattern_matchs

see some sample data.

In [17]:
def show_entity_label(text_entities, pattern_matchs):
    for example in text_entities:
        print('-'*50)
        text, annotations = example
        print(text) 
        print(*[(text[start:end], start, end, label) for start, end, label in annotations['entities']], sep='\n')
    print('='*50)
    print(*pattern_matchs, sep='\n')
    
texts = [standardize_text1(note) for note in df_sample.notes]  
sample_text_entities, sample_pattern_matchs = generate_label(texts, patterns=patterns)
show_entity_label(sample_text_entities, sample_pattern_matchs)

--------------------------------------------------
please create a revisit for wednesday . #normal please order part # l51580-001 and l31365-001 to concord , california hal setting up time to meet with customer . 
('l51580-001', 68, 78, 'PART_NUM')
('l31365-001', 83, 93, 'PART_NUM')
--------------------------------------------------
please order battery 933321-855 ship to ups hold for pickup , tuscany way , austin . . please reassign to rob silcocks
('933321-855', 21, 31, 'PART_NUM')
--------------------------------------------------
per l2 please order: 2yb64-67005 , 2yb64-67003 xl3600 , 6kd23-67021 xl3600 , 6kd23-67030 , and two 2yb64-67002
('2yb64-67005', 21, 32, 'PART_NUM')
('2yb64-67003', 35, 46, 'PART_NUM')
('6kd23-67021', 56, 67, 'PART_NUM')
('6kd23-67030', 77, 88, 'PART_NUM')
('2yb64-67002', 99, 110, 'PART_NUM')
--------------------------------------------------
please create a revisite for 7-16-2021 at 1300 order part number q5669-60687 please order 2 q6683-67001 t8w18-67001 s

In [18]:
texts = [standardize_text1(note) for note in df_train_test.notes]            
text_entities, pattern_matchs = generate_label(texts, patterns=patterns)

In [19]:
print(*pattern_matchs, sep='\n')  

('[a-zA-Z][a-zA-Z]\\d\\d\\d-\\d\\d\\d\\d\\d', 1449)
('[a-zA-Z]\\d[a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d', 1089)
('[a-zA-Z][a-zA-Z]\\d-\\d\\d\\d\\d-\\d\\d\\d[a-zA-Z][a-zA-Z]', 643)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d[a-zA-Z]\\d\\d\\d\\d', 597)
('[a-zA-Z]\\d\\d\\d\\d\\d-\\d\\d\\d', 547)
('\\d\\d\\d\\d\\d\\d-\\d\\d\\d', 400)
('[a-zA-Z][a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d[a-zA-Z]', 271)
('\\d\\d\\d\\d-\\d\\d\\d\\d', 166)
('[a-zA-Z]\\d\\d\\d\\d-\\d\\d\\d\\d\\d', 122)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d[a-zA-Z]\\d[a-zA-Z]\\d\\d', 46)
('\\d[a-zA-Z][a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d', 39)
('[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\\d\\d\\d[a-zA-Z]\\d\\d[a-zA-Z]', 21)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d\\d[a-zA-Z][a-zA-Z]\\d\\d\\d\\d\\d', 14)
('[a-zA-Z]\\d\\d\\d\\d\\d-[a-zA-Z][a-zA-Z]\\d', 6)
('\\d\\d-\\d\\d\\d\\d\\d-\\d\\d\\d', 5)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d[a-zA-Z]\\d[a-zA-Z][a-zA-Z]\\d', 5)
('[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d[a-zA-Z]', 5)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d\\d[a-zA-Z]\\d\

In [20]:
indexes = np.random.randint(0, len(text_entities), 10)
sample_text_entities = [text_entities[i] for i in indexes]
sample_pattern_matchs = pattern_matchs[0:len(indexes)]
show_entity_label(sample_text_entities, sample_pattern_matchs)

--------------------------------------------------
l0h24a-67903 , 1 , c2eges21

--------------------------------------------------
laptop has a video issue that support asked us to look at . 830 g7 - sn: 5cg1048xk2 . 

--------------------------------------------------
need revisit for 7/2 and please order 0609-001558 qty 1 nbd to 83005433

--------------------------------------------------
order 2 q7404-60029 hinge . please order cf116-67918 customer ok with waiting , part stuck in memphis 7-15
('q7404-60029', 8, 19, 'PART_NUM')
('cf116-67918', 41, 52, 'PART_NUM')
--------------------------------------------------
please order l0h24-67903 from c2b9 for pick up
('l0h24-67903', 13, 24, 'PART_NUM')
--------------------------------------------------
please create a revisit for this case . 

--------------------------------------------------
pls change primary contact to ann woodruff 9543784520 ann . m . woodruff@questdiagnostics . com . ce rescheduling because customer unreachable because

In [21]:
train_data, test_data = train_test_split(text_entities, test_size=0.25, random_state=99)

print(len(train_data))
print(len(test_data))

4582
1528


### Generate NER data

In [22]:
def generate_data(data):
    nlp = spacy.blank("en") # load a new spacy model
    db = DocBin() # create a DocBin object

    for text, annot in tqdm(data): # data in previous format
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        for start, end, label in annot["entities"]: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents # label the text with the ents
        db.add(doc)
    return db
    
train_spacy = generate_data(train_data)
train_spacy.to_disk("./train.spacy") # save the docbin object
test_spacy = generate_data(test_data) 
test_spacy.to_disk("./test.spacy") # save the docbin object

100%|██████████| 1528/1528 [00:01<00:00, 1197.72it/s]


In [23]:
!ls -l . 

total 7620
-rw-r--r--. 1 root root    1916 Aug 23 01:30 base_config.cfg
-rw-r--r--. 1 root root    1664 Aug 23 09:48 base_config_gpu.cfg
-rw-r--r--. 1 root root    2836 Aug 24 15:47 config.cfg
-rw-r--r--. 1 root root    2625 Aug 24 15:48 config_gpu.cfg
-rw-r--r--. 1 root root  111810 Aug 24 16:14 custom_ner.ipynb
-rw-r--r--. 1 root root  261717 Aug 24 15:54 custom_ner_aug.ipynb
drwxr-xr-x. 5 root root      43 Aug 24 09:04 output
-rw-r--r--. 1 root root  227413 Aug 24 16:15 test.spacy
-rw-r--r--. 1 root root  229957 Aug 24 15:04 test_aug.spacy
-rw-r--r--. 1 root root  660076 Aug 24 16:15 train.spacy
-rw-r--r--. 1 root root 6281781 Aug 24 15:04 train_aug.spacy


## Train the model

In [24]:
# !python -m spacy train config.cfg --output ./output/cpu --paths.train ./train.spacy --paths.dev ./test.spacy

In [48]:
!python -m spacy train config_gpu.cfg --output ./output/gpu --paths.train ./train.spacy --paths.dev ./test.spacy --gpu-id 0

2021-08-25 00:08:19.450614: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2021-08-25 00:08:22,850] [INFO] Set up nlp object from config
[2021-08-25 00:08:22,867] [INFO] Pipeline: ['transformer', 'ner']
[2021-08-25 00:08:22,873] [INFO] Created vocabulary
[2021-08-25 00:08:22,874] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are init

## Evaluate

###  Check test data

In [49]:
nlp = spacy.load("output/gpu/model-last")

In [50]:
options = {'colors': {'ACTION':"#56D7C4", 'PART_NUM':"#92E0AA"} }

def show_text(text, annotations=None, options=options, show_detail=True, standardize_text=None):
    if standardize_text is not None:
        text = standardize_text(text)
    doc = nlp(text)
    print('='*100)
    if show_detail:
        print(doc.text) 
    spacy.displacy.render(doc, style='ent', options=options)
    if show_detail and annotations is not None and len(annotations['entities'])>0:
        print('-'*25, 'acutal entities', '-'*25)
        print(*[(text[start:end], start, end, label) for start, end, label in annotations['entities']], sep='\n')   
    if show_detail and len(doc.ents)>0:
        print('-'*25, 'predict entities', '-'*25)
        print(*[(ent.text, ent.start_char, ent.end_char, ent.label_) for ent in doc.ents], sep='\n')      

def show_texts(data, indexes=None, options=options, show_detail=True, standardize_text=None):
    def show_text_(data):
        if isinstance(data, str):
            show_text(data, None, options=options, show_detail=show_detail, standardize_text=standardize_text)
        else:
            text, annotations = data
            show_text(text, annotations, options=options, show_detail=show_detail, standardize_text=standardize_text)  
            
    if indexes is not None:
        for i in indexes:
            show_text_(data[i])   
    else:
        for one_data in data: 
            show_text_(one_data)      
    
                  

In [51]:
indexes = np.random.randint(0, len(test_data), 10)
show_texts(test_data, indexes) 

hello please order part j9v82-67001 serv assy-main pca - j9v82 , and ship it to raleigh fedex hfpu . i ' m rescheduling due to routine optimization


please order part #: rm2-6613-000cn and ship to hal


------------------------- acutal entities -------------------------
('rm2-6613-000cn', 21, 35, 'PART_NUM')
------------------------- predict entities -------------------------
('rm2-6613-000cn', 21, 35, 'PART_NUM')
the adapter as not arrived yet but i have been told that the computer is back online now and can be cancelled . please reassign this case to james lacy . he is expecting it . thank you . waiting on an adapter to be able to do this reimage . 


request revisit to order part


please create follow up for tomorrow and order part nbd to hal in windsor locks ct . b3q10-40081 qty1 , b3q10-40080 qty1 . #normal monday is a holiday so nbd is wednesday for a call coming in on weekend . 


------------------------- acutal entities -------------------------
('b3q10-40081', 85, 96, 'PART_NUM')
('b3q10-40080', 104, 115, 'PART_NUM')
------------------------- predict entities -------------------------
('b3q10-40081', 85, 96, 'PART_NUM')
('b3q10-40080', 104, 115, 'PART_NUM')
please order the recommended parts to my hfpu nbd


please create a revisit and order part b5l47-67907 and ship to palm springs ca . 


------------------------- acutal entities -------------------------
('b5l47-67907', 39, 50, 'PART_NUM')
------------------------- predict entities -------------------------
('b5l47-67907', 39, 50, 'PART_NUM')
reassign to rick ficek he requested this case also reorder 933321-852 battery and ship to 321 ellendale parkway crown point , in 46307


------------------------- acutal entities -------------------------
('933321-852', 59, 69, 'PART_NUM')
------------------------- predict entities -------------------------
('933321-852', 59, 69, 'PART_NUM')
#normal customer requests service at this time order 917725-855 and l43407-001 and 213349-001 for next day ship to customers site attn kim gordin thank you


------------------------- acutal entities -------------------------
('917725-855', 53, 63, 'PART_NUM')
('l43407-001', 68, 78, 'PART_NUM')
('213349-001', 83, 93, 'PART_NUM')
------------------------- predict entities -------------------------
('917725-855', 53, 63, 'PART_NUM')
('l43407-001', 68, 78, 'PART_NUM')
('213349-001', 83, 93, 'PART_NUM')
#normal please reassign ticket to tech laneisha anderson , as she will be covering this while i am out on vacation . 


In [52]:
show_texts(test_data, indexes, show_detail=False) 

### Check Error Data

In [62]:
def get_error_data(data):
    error_data = []
    for i in range(len(data)):
        text, annotations = data[i]
        doc = nlp(text)
        entities = annotations['entities']
        if len(entities) != len(doc.ents):
            error_data.append(data[i])
            print(i, len(error_data))
        else:
            exit_flag = False
            actual_entities = {(start, end, label): label for start, end, label in entities}
            predict_entities = {(ent.start_char, ent.end_char, ent.label_): ent.label_ 
                                for ent in doc.ents}
            
            for key, _ in actual_entities.items():                
                if key not in predict_entities:
#                     print(actual_entities, predict_entities, sep='\n') 
                    error_data.append(data[i])
                    print(i, len(error_data))
                    break
            if exit_flag: continue
            for key, _ in predict_entities.items():
                if key not in actual_entities:
#                     print(actual_entities, predict_entities)
                    error_data.append(data[i])
                    print(i, len(error_data))
                    break
                
    return error_data

error_data = get_error_data(test_data)

0 1
25 2
116 3
121 4
165 5
198 6
207 7
235 8
510 9
576 10
699 11
763 12
763 13
765 14
881 15
914 16
949 17
958 18
967 19
1006 20
1068 21
1070 22
1070 23
1126 24
1141 25
1141 26
1196 27
1202 28
1209 29
1266 30
1299 31
1308 32
1325 33
1349 34
1374 35
1416 36
1484 37
1517 38
1517 39


In [54]:
print(len(error_data))

39


In [55]:
show_texts(error_data)  

order parts jc90-01679a , jc90-01680a , jc93-01063c , 6602-003185 quantity of 1 each and ship to my hold for pickup via fedex . . #normal osa met rdt


------------------------- acutal entities -------------------------
('jc90-01679a', 12, 23, 'PART_NUM')
('jc90-01680a', 26, 37, 'PART_NUM')
('jc93-01063c', 40, 51, 'PART_NUM')
------------------------- predict entities -------------------------
('jc90-01679a', 12, 23, 'PART_NUM')
('jc90-01680a', 26, 37, 'PART_NUM')
('jc93-01063c', 40, 51, 'PART_NUM')
('6602-003185', 54, 65, 'PART_NUM')
please send pn933321-855 to 2825route 18south old bridge nj 08857 att aleksandr kamsan


------------------------- predict entities -------------------------
('pn933321-855', 12, 24, 'PART_NUM')
need to order nib kit and ecu ( jz09-67077 & g1w39-67001 ) ship to san antonio hal


------------------------- acutal entities -------------------------
('g1w39-67001', 45, 56, 'PART_NUM')
------------------------- predict entities -------------------------
('jz09-67077', 32, 42, 'PART_NUM')
('g1w39-67001', 45, 56, 'PART_NUM')
please clone and transfer the part to the cloned case . . #normal please order jc97-04650a to the baton rouge hal . mike try these site contacts .   haleigh – 225-663-4041 – haleigh_a_winston kayla – 225-663-4031 – kayla_r_naquin


------------------------- acutal entities -------------------------
('jc97-04650a', 79, 90, 'PART_NUM')
------------------------- predict entities -------------------------
('jc97-04650a', 79, 90, 'PART_NUM')
('–\xa0', 157, 159, 'PART_NUM')
('– haleigh_a_winston', 172, 191, 'PART_NUM')
('–\xa0', 198, 200, 'PART_NUM')
('– kayla_r_naquin', 213, 229, 'PART_NUM')
please order parts below: g1w41-67908/ kit-adf whole unit cn598-67018 / separator / pick assy kit g1w39-67952 / kit-assy/feeder tray 3 g1w39-67953 / kit-assy/feeder tray 4 g1w39-67954 / kit-assy/feeder tray 5 please ship to client address below: sm


...

------------------------- predict entities -------------------------
('2-1235-000cn', 18, 30, 'PART_NUM')
('3x550-sheet', 31, 42, 'PART_NUM')
lex-c930x76gh , 1 , c2efes25 lex-c930x73g , 1 , c2efes25


------------------------- acutal entities -------------------------
('lex-c930x73g', 29, 41, 'PART_NUM')
------------------------- predict entities -------------------------
('lex-c930x76gh ,', 0, 15, 'PART_NUM')
('lex-c930x73g', 29, 41, 'PART_NUM')
b of a order the following for nbd: jc39-02567a , jc39-02553a , jc39-02584 , and ( 2 ) 5851-6712 ship to my primary hal


------------------------- acutal entities -------------------------
('jc39-02567a', 36, 47, 'PART_NUM')
('jc39-02553a', 50, 61, 'PART_NUM')
('5851-6712', 87, 96, 'PART_NUM')
------------------------- predict entities -------------------------
('jc39-02567a', 36, 47, 'PART_NUM')
('jc39-02553a', 50, 61, 'PART_NUM')
('jc39-02584', 64, 74, 'PART_NUM')
('5851-6712', 87, 96, 'PART_NUM')
please send part to local hal in louisville , ky 795970-602 . please send part to local hal in louisville , kentucky 795970-602


------------------------- acutal entities -------------------------
('795970-602', 49, 59, 'PART_NUM')
('795970-602', 117, 127, 'PART_NUM')
------------------------- predict entities -------------------------
('ky 795970-602', 46, 59, 'PART_NUM')
('795970-602', 117, 127, 'PART_NUM')
please send part to local hal in louisville , ky 795970-602 . please send part to local hal in louisville , kentucky 795970-602


------------------------- acutal entities -------------------------
('795970-602', 49, 59, 'PART_NUM')
('795970-602', 117, 127, 'PART_NUM')
------------------------- predict entities -------------------------
('ky 795970-602', 46, 59, 'PART_NUM')
('795970-602', 117, 127, 'PART_NUM')


In [56]:
doc = nlp('rm26322-000cn PART_NUM e6b69-67901 PART_NUM rm1-8413-000cn PART_NUM please send ACTION to fedex ACTION 3242 herrman rd garland , texas 75041 on 2021-08-21')
print(*[(ent.text, ent.start_char, ent.end_char, ent.label_) for ent in doc.ents], sep='\n') 
spacy.displacy.render(doc, style='ent', options=options)

('rm26322-000cn', 0, 13, 'PART_NUM')
('e6b69-67901', 23, 34, 'PART_NUM')
('rm1-8413-000cn', 44, 58, 'PART_NUM')


### Some Features and Issues

#### Miss some part-numbers 

In [57]:
sample_parts = ['M14302-161', '926537-001', 'M46915-D01', '140314-FB2', '176253-8B0', 
               '442010-ABC', 'L07896-DB1', '168757-B21', 'FE-15520-01', 'RM1-7867-000CN', 
               '5066-4719', '5066-3872-1', '3HZ90-30001', 'T6M22-30001', '98544-04105', 
               'C8109-69018', 'L26480-DB1', '632427-001B', 'CN463-69003', 'RM1-1298BULK', 
               '8-752-078-46', 'X-4035-119-1', 'TBLB3002-A04', '4822-701-15319', '08-2212C-33W', 
               '19-40065-011', 'ATT09850-66532', '98564-66572-SQ', '897-250144AA', '738-576-94', 
               '1FQ6-0001', '1818-7469-LGSA', '1820-6361M', '1818-6868-SAM', '3138-107-96160', 
               '686858R-999', 'MD30C-A2', '29-26196-00', 'DEL-7D092', 'LEX-40X2384', 'LEX-40X2831H', 
               'SAM-6107001172', 'SHA-1625DS51', 'XER-006R01275', 'BRO-TN115BK', 'DEL-310-5811', 
               'DEL-PK496', 'KYO-1702F97US0', 'LA95-CA', 'KYO-DK-310H', 'EPS-C31CD38A9921', 
               'KYO-F994091H', 'EPS-CEPS-003G', 'LEX-24015SA', 'LEX-62D1X00', 'LEX-72K0DV0', 
               'LEX-801HC', 'LEX-C5220CSH', 'TRO-78-24619-001', 'LEX-C544X1CG', 'HPE-749797-001', 
               'SFT-P-895', 'LEX-E260A21A-C', 'RIC-D0296509', 'SEA-ST1000LM049', 'LEX-12A8400-PC', 
               'HPE-JL258A', 'RIC-400507', 'FUJ-FPCPR362AQ', 'TOS-6LE8296100', 'XER-003K04980', 
               'XER-116111500', 'ATS-2711FXSC-901', 'BRO-TN336BK', 'DEL-310-5807', 'HPI-CF258A-M', 
               'LEX-T101-0000000', 'DEL-9PN5P', 'SAM-BA92-08880A', 'APC-RBC7', 'KMH-4448-121', 
               'SXC-C7ABTTAAB', 'TRO-04621201', 'VEF-28924-04-R', 'WFP-HPM527Z', 'KYO-1702LK0UN2', 
               'APC-SUA1500RM2U', 'KYO-302NM18021', 'HPE-JH329-61001', 'KYO-1T02GA0US0', 'LEB-04X4674',
               'KYO-302GR93034', 'DEL-V1RX3', 'LEX-40X0593', 'TOS-6LH3460800', 'RIC-AE020171', 
               'XER-59K60140', 'BRO-LM5140001', 'KMH-A00JA56600', 'KYO-302F906240', 'BRO-LEM084001', 
               'XER-116-2035', 'BRO-BU100CLH', '3X-LK465-A2', '4G1-3999-030CN', 'H3980-60002BULK', 
               'SHA-VHI0MFP000', 'TOS-C017839000', 'JC66-01190A', 'XER-3335DNI', 'XER-3635MFP-S', 
               'XER-4622DN', 'XER-7750-Z2', 'DEL-D1320-W3', 'DEL-P637D', 'LEX-C540X35G', 
               'SHA-CPLTM8190F', 'SHACFRM-1380DS53', 'XER-401-0855-0', '90-0077', 'BRO-LF6710001', 
               'Q3938-68001-PCA', 'C314X+49A-90002', '20ER-84530KC', 'RM2-2903-BULK', 'CE506-67919-BU', 
               'CH971-91596', 'DEL-FM235', 'JC44-00210E', 'SHACFRM-1509DS52', 'SHADHAI-5043FCPZ', 
               'SHAGCAB-1102FCB1', '000-50-02-008', '504000226-DD', 'SHAVHPLG217L5A-1', 'SHAVHPOSH03Y02-1', 
               'SHADUNT-8962FCP1', 'SHAKI-OK0010FCPZ', 'SHAQSW-P0506FCZZ', 'SHAVHPGP1A73AR-1', 'SHARH-IX0055FCPZ', 
               '358-000001', 'SHAVHPGP1A71L3-18', 'B23-0994', '504000226-D', '5066-0731', 
               'E0-CABLE-01', 'RM1-8508-010-BU', 'B3M77-67902-BU', 'CA02626-E029FJ', 'CF065-67901-BU', 
               'C2H57-67901-BULK', 'LN08-A2', '693709-001B', 'SS467-67001', 'LEX-70C0D20', 
               'SHAQSW-M0518FCPZ', 'SHALX-BZ0994FCPZ', 'LN08X-TF', 'SHAGCAB-1507FCAZ']

sample_parts = [part.lower() for part in sample_parts]

In [59]:
def get_texts(parts, standardize_text=standardize_text1):
    texts = [standardize_text(f'Please order qty 1 {part} NBD to my HAL in Lenexa. Thank you.') for part in parts]
    return texts

def check_sample_parts(parts):  
    texts = get_texts(parts)
    docs = nlp.pipe(texts)
    texts = []
    error_texts = []
    errpr_parts = []
    for doc, part in zip(docs, parts):
        texts.append(doc.text)
        if len(doc.ents) !=1 or doc.ents[0].text != part.lower():
            error_texts.append(doc.text)
            errpr_parts.append(part)
    return texts, error_texts, errpr_parts
        
parts = ['000-50-02-008']
texts, error_texts, errpr_parts = check_sample_parts(parts)
show_texts(texts, show_detail=True) 

please order qty 1 000-50-02-008 nbd to my hal in lenexa . thank you . 


------------------------- predict entities -------------------------
('000-50-02', 19, 28, 'PART_NUM')


In [61]:
parts = sample_parts
texts, error_texts, errpr_parts = check_sample_parts(parts)
print(f'find {len(error_texts)}/{len(parts)} errors')
show_texts(error_texts, show_detail=True)  

find 89/160 errors
please order qty 1 8-752-078-46 nbd to my hal in lenexa . thank you . 


please order qty 1 x-4035-119-1 nbd to my hal in lenexa . thank you . 


------------------------- predict entities -------------------------
('x-4035-119', 19, 29, 'PART_NUM')
please order qty 1 tblb3002-a04 nbd to my hal in lenexa . thank you . 


please order qty 1 4822-701-15319 nbd to my hal in lenexa . thank you . 


------------------------- predict entities -------------------------
('4822-701', 19, 27, 'PART_NUM')
please order qty 1 08-2212c-33w nbd to my hal in lenexa . thank you . 


...

------------------------- predict entities -------------------------
('b3m77-67902', 19, 30, 'PART_NUM')
please order qty 1 cf065-67901-bu nbd to my hal in lenexa . thank you . 


------------------------- predict entities -------------------------
('cf065-67901', 19, 30, 'PART_NUM')
please order qty 1 c2h57-67901-bulk nbd to my hal in lenexa . thank you . 


------------------------- predict entities -------------------------
('c2h57-67901', 19, 30, 'PART_NUM')
please order qty 1 shaqsw-m0518fcpz nbd to my hal in lenexa . thank you . 


please order qty 1 shalx-bz0994fcpz nbd to my hal in lenexa . thank you . 


please order qty 1 shagcab-1507fcaz nbd to my hal in lenexa . thank you . 


In [ ]:
error_text_entities, error_pattern_matchs = generate_label(error_texts, patterns=patterns)
show_entity_label(error_text_entities, error_pattern_matchs)

I think that the issues are mainly caused by lacking of these parts in training data. 

There are 2 solutions:

1. mmerge some rules from regular expressions and others.
2. use data augmentation to generate a lot of random part numbers based on the rules.

####  Find potentail part number

In [ ]:
parts = ['FE-15520-01', 'FE-15520-012', 'FE-15520-01233']
texts, error_texts, errpr_parts = check_sample_parts(parts)
print(f'find {len(error_texts)}/{len(parts)} errors')
show_texts(texts, show_detail=True)  


In [ ]:
some_text_entities, sample_pattern_matchs = generate_label(texts, patterns=patterns)
show_entity_label(sample_text_entities, sample_pattern_matchs)

#### find potential actions 

it has some effects, but not good enough.

In [ ]:
texts = ['order qty 1 fe-15520-01 nbd to my hal in lenexa . thank you . ',
         'orders qty 1 fe-15520-01 nbd to my hal in lenexa . thank you . ',
         'send qty 1 fe-15520-01 nbd to my hal in lenexa . thank you . ',
         'sends qty 1 fe-15520-01 nbd to my hal in lenexa . thank you . ',
         'ship qty 1 fe-15520-01 nbd to my hal in lenexa . thank you . ',
         'ships qty 1 fe-15520-01 nbd to my hal in lenexa . thank you . 2012-02-02',         
        ]

show_texts(texts, show_detail=False) 